In [1]:
import re
import scrapy
from scrapy.crawler import CrawlerProcess
from urllib.request import urlretrieve

In [2]:
OUTPUT_FOLDER = './raw_data/'

In [3]:
base_url = 'https://www.chessgames.com'
start_url = 'https://www.chessgames.com/perl/chesscollection?cid=1014770'

In [4]:
class ChessGamesSpider(scrapy.Spider):
    name = 'chess-games'
    start_urls = [start_url]

    base_url = base_url
    output_folder = OUTPUT_FOLDER
    
    def parse_game(self, response):
        links = response.css('td > font > a::attr(href)').extract()
        pgn_link = list(filter(lambda link: link.startswith('/pgn/'), links))[0]
        
        pat = '(?:\/pgn\/)(.*)(?:.pgn\?gid=)(\d+)'
        fname, gid = re.search(pattern=pat, string=pgn_link).groups()
        
        urlretrieve(url=self.base_url + pgn_link,
                    filename=self.output_folder + fname + '_' + gid + '.pgn')
    
    def parse(self, response):
        links = response.css('p > font > a::attr(href)').extract()
        for link in links:
            yield scrapy.Request(url=self.base_url+link,
                                 callback=self.parse_game,
                                 dont_filter=True)


In [5]:
process = scrapy.crawler.CrawlerProcess()
process.crawl(ChessGamesSpider)
process.start()

2020-08-29 16:52:00 [scrapy.utils.log] INFO: Scrapy 2.0.1 started (bot: scrapybot)
2020-08-29 16:52:00 [scrapy.utils.log] INFO: Versions: lxml 4.3.2.0, libxml2 2.9.9, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 19.2.0, Python 3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1c  28 May 2019), cryptography 2.6.1, Platform Windows-10-10.0.17134-SP0
2020-08-29 16:52:00 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-08-29 16:52:00 [scrapy.crawler] INFO: Overridden settings:
{}
2020-08-29 16:52:00 [scrapy.extensions.telnet] INFO: Telnet Password: 5374bf00805ac3a4
2020-08-29 16:52:00 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2020-08-29 16:52:01 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'sc